In [1]:
import pandas as pd
import torch
import torch.nn as nn
from torch.utils.data import DataLoader, Dataset
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import MinMaxScaler
from sklearn.metrics import mean_squared_error
import numpy as np
import tqdm as tqdm
import torch.optim as optim
import os

Kaggle Data set
https://www.kaggle.com/datasets/ronakbadhe/chess-evaluations

In [2]:
df = pd.read_csv('Data/chessData.csv')

In [3]:
print(df.head())
df = df[:(len(df)//32)].copy()
df = df.dropna()
print(len(df))


                                                 FEN Evaluation
0  rnbqkbnr/pppppppp/8/8/4P3/8/PPPP1PPP/RNBQKBNR ...        -10
1  rnbqkbnr/pppp1ppp/4p3/8/4P3/8/PPPP1PPP/RNBQKBN...        +56
2  rnbqkbnr/pppp1ppp/4p3/8/3PP3/8/PPP2PPP/RNBQKBN...         -9
3  rnbqkbnr/ppp2ppp/4p3/3p4/3PP3/8/PPP2PPP/RNBQKB...        +52
4  rnbqkbnr/ppp2ppp/4p3/3p4/3PP3/8/PPPN1PPP/R1BQK...        -26
404938


In [4]:
def clean_evals(entry:str):
    if entry[0] == '#':
        side = 1000 if (entry[1] == '+') else -1000
        return int(entry[2:]) + side
    else:
        return int(entry)



In [5]:
df['Evaluation'] = df['Evaluation'].apply(clean_evals)

In [6]:
def convert_to_index(pos: str) -> int:
    if pos == "-":
        return 0
    file_map = {value: key for key, value in enumerate("abcdefgh")}
    col = file_map[pos[0]]
    row = int(pos[1]) - 1
    # print(f"col: {col}, row: {row}")
    return 1 << (row * 8) + col


In [7]:

# Convert binary vectors to PyTorch tensors
# tensor_data = torch.tensor(binary_vectors, dtype=torch.float32)
def convert_pieces_to_vectors(piece_map: dict[str: int]) -> None:
    for piece, board in piece_map.items():
        piece_map[piece] = np.array([(board >> i) & 1 for i in range(64)], dtype=np.float32)

In [8]:

def convert_fen(fen: str):
    piece_map = {value: 0 for key, value in enumerate("KQRBNPkqrbnp")}
    # print(f"Piece_Map: {piece_map}")
    ranks = fen.split("/")
    # print(f"ranks: {ranks}")
    curr_rank = 8
    while curr_rank > 0:
        r = ranks[8-curr_rank]
        file_index = 0
        # print(f"rank: {curr_rank-1} | {r}")
        for i in range(len(r)):
            c = r[i]
            if file_index >= 8:
                convert_pieces_to_vectors(piece_map)
                states = r[i:].split(" ")[1:]
                # print(f"States: {states}")
                piece_map["moves"] = 0 if states[0] == 'w' else 1
                piece_map['WhiteKing'] = 1 if "K" in states[1] else 0
                piece_map['WhiteQueen'] = 1 if 'Q' in states[1] else 0
                piece_map['BlackKing'] = 1 if 'k' in states[1] else 0
                piece_map['BlackQueen'] = 1 if 'q' in states[1] else 0
                piece_map["en_passant"] =  np.array([(convert_to_index(states[2]) >> i) & 1 for i in range(64)], dtype=np.float32)
                piece_map["half_clock"] = int(states[3])
                piece_map["full_clock"] = int(states[4])
                return piece_map
            else:
                if c.isdigit():
                    file_index += int(c)
                else:
                    # print(f"c: {c}")
                    piece_map[c] += (1 << ((curr_rank * 8) + file_index))
                    file_index += 1
        curr_rank -= 1
    return piece_map
                


In [9]:
df['Longs'] = df['FEN'].apply(convert_fen)
df = pd.concat([df, pd.json_normalize(df['Longs'])], axis=1)
df = df.drop(['FEN', 'Longs'], axis=1)
print(df.head())

   Evaluation                                                  K  \
0         -10  [0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...   
1          56  [0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...   
2          -9  [0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...   
3          52  [0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...   
4         -26  [0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...   

                                                   Q  \
0  [0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...   
1  [0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...   
2  [0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...   
3  [0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...   
4  [0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...   

                                                   R  \
0  [0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 1.0, ...   
1  [0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 1.0, ...   
2  [0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 1.0, ...   
3  [0.0, 0.0, 0.0, 0.0, 0.0, 0

In [10]:
print(len(df['K']))
print(len(df['en_passant']))

404938
404938


In [12]:
print(df.columns)

Index(['Evaluation', 'K', 'Q', 'R', 'B', 'N', 'P', 'k', 'q', 'r', 'b', 'n',
       'p', 'moves', 'WhiteKing', 'WhiteQueen', 'BlackKing', 'BlackQueen',
       'en_passant', 'half_clock', 'full_clock'],
      dtype='object')


In [11]:
# Print out the dimensions and types of all relevant columns
for col in ['K', 'Q', 'R', 'B', 'N', 'P', 'k', 'q', 'r', 'b', 'n', 'p', 'en_passant']:
    print(f"{col}: Type: {type(df[col].iloc[0])}, Shape: {np.array(df[col].iloc[0]).shape}")

# Ensure 'en_passant' is an array
df['en_passant'] = df['en_passant'].apply(lambda x: np.array(x, dtype=np.float32) if not isinstance(x, np.ndarray) else x)

# Recheck dimensions and types after conversion
for col in ['K', 'Q', 'R', 'B', 'N', 'P', 'k', 'q', 'r', 'b', 'n', 'p', 'en_passant']:
    print(f"{col}: Type: {type(df[col].iloc[0])}, Shape: {np.array(df[col].iloc[0]).shape}")


K: Type: <class 'numpy.ndarray'>, Shape: (64,)
Q: Type: <class 'numpy.ndarray'>, Shape: (64,)
R: Type: <class 'numpy.ndarray'>, Shape: (64,)
B: Type: <class 'numpy.ndarray'>, Shape: (64,)
N: Type: <class 'numpy.ndarray'>, Shape: (64,)
P: Type: <class 'numpy.ndarray'>, Shape: (64,)
k: Type: <class 'numpy.ndarray'>, Shape: (64,)
q: Type: <class 'numpy.ndarray'>, Shape: (64,)
r: Type: <class 'numpy.ndarray'>, Shape: (64,)
b: Type: <class 'numpy.ndarray'>, Shape: (64,)
n: Type: <class 'numpy.ndarray'>, Shape: (64,)
p: Type: <class 'numpy.ndarray'>, Shape: (64,)
en_passant: Type: <class 'numpy.ndarray'>, Shape: (64,)
K: Type: <class 'numpy.ndarray'>, Shape: (64,)
Q: Type: <class 'numpy.ndarray'>, Shape: (64,)
R: Type: <class 'numpy.ndarray'>, Shape: (64,)
B: Type: <class 'numpy.ndarray'>, Shape: (64,)
N: Type: <class 'numpy.ndarray'>, Shape: (64,)
P: Type: <class 'numpy.ndarray'>, Shape: (64,)
k: Type: <class 'numpy.ndarray'>, Shape: (64,)
q: Type: <class 'numpy.ndarray'>, Shape: (64,)
r: T

In [13]:
# Example DataFrame structure
# df contains columns like 'K', 'Q', 'R', etc. for piece positions
# and additional columns for 'moves', 'half_clock', 'full_clock', and castling rights

def prepare_input(row):
    # Extract the binary vectors for each piece type
    channels = []
    for piece in ['K', 'Q', 'R', 'B', 'N', 'P', 'k', 'q', 'r', 'b', 'n', 'p', 'en_passant']:
        piece_vector = np.array(row[piece]).reshape(8, 8)
        channels.append(piece_vector)
    
    # Additional channel for castling rights
    castling_rights_channel = np.zeros((8, 8))
    if row['WhiteKing']:
        castling_rights_channel += 1
    if row['WhiteQueen']:
        castling_rights_channel += 2
    if row['BlackKing']:
        castling_rights_channel += 4
    if row['BlackQueen']:
        castling_rights_channel += 8
    channels.append(castling_rights_channel)
    
    # Additional channels for 'moves', 'half_clock', and 'full_clock'
    moves_channel = np.full((8, 8), row['moves'])
    half_clock_channel = np.full((8, 8), row['half_clock'])
    full_clock_channel = np.full((8, 8), row['full_clock'])
    channels.extend([moves_channel, half_clock_channel, full_clock_channel])
    
    input_tensor = np.stack(channels)
    return input_tensor

# Apply this function to all rows to get the input data for the CNN
df['input_tensor'] = df.apply(prepare_input, axis=1)


In [12]:
# def concatenate_vectors(row):
#     return np.concatenate([row['K'], row['Q'], row['R'], row['B'], row['N'], row['P'], row['k'], row['q'], row['r'], row['b'], row['n'],row['p'], row['en_passant']])

# df['board_state'] = df.apply(concatenate_vectors, axis=1)

In [16]:
# X = pd.concat([df['board_state'].apply(pd.Series), df[['moves', 'half_clock', 'full_clock', 
#       'WhiteKing', 'WhiteQueen', 'BlackKing', 'BlackQueen']]], axis=1)
X = [torch.tensor(x, dtype=torch.float32) for x in df['input_tensor']]
y = df['Evaluation']
eval_scaler = MinMaxScaler(feature_range=(-1, 1))
y = eval_scaler.fit_transform(y.values.reshape(-1, 1))
# nfeats = len(X.columns)
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

In [17]:
class Chess_Dataset(Dataset):
    def __init__(self, X, y):
        self.X = X
        self.y = torch.tensor(y, dtype=torch.float32)
    def __len__(self):
        return len(self.y)
    def __getitem__(self, idx):
        return self.X[idx], self.y[idx]

In [36]:
BATCH_SIZE = 32
CKPT_DIR = "CKPT_CNN"
LR = 0.001
EPOCHS = 1000

In [19]:
print(X_train)
print(y_train)

KeyboardInterrupt: 

In [37]:
train_dataset = Chess_Dataset(X_train, y_train)
train_loader = DataLoader(train_dataset, batch_size=BATCH_SIZE, shuffle=True)
test_dataset = Chess_Dataset(X_test, y_test)
test_loader = DataLoader(test_dataset, batch_size=BATCH_SIZE, shuffle=True)

In [21]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print(device)

cuda


In [38]:
class Chess_CNN(nn.Module):
    def __init__(self, input_channels=17, num_classes=1):
        super(Chess_CNN, self).__init__()
        self.conv1 = nn.Conv2d(input_channels, 32, kernel_size=3, stride=1, padding=1)
        self.conv2 = nn.Conv2d(32, 64, kernel_size=3, stride=1, padding=1)
        self.conv3 = nn.Conv2d(64, 128, kernel_size=3, stride=1, padding=1)
        self.fc1 = nn.Linear(128 * 1 * 1, 512)  # Corrected input size
        self.fc2 = nn.Linear(512, num_classes)

    def forward(self, x):
        x = torch.relu(self.conv1(x))
        x = torch.max_pool2d(x, kernel_size=2, stride=2)
        x = torch.relu(self.conv2(x))
        x = torch.max_pool2d(x, kernel_size=2, stride=2)
        x = torch.relu(self.conv3(x))
        x = torch.max_pool2d(x, kernel_size=2, stride=2)
        x = x.view(x.size(0), -1)
        x = torch.relu(self.fc1(x))
        x = self.fc2(x)
        return x


In [49]:
class Chess_Model(nn.Module):
    def __init__(self, nfeats):
        super(Chess_Model, self).__init__()
        self.input_layer = nn.Linear(nfeats, 64)
        self.lyr1 = nn.Linear(64, 128)
        self.lyr2 = nn.Linear(128, 64)
        self.lyr3 = nn.Linear(64, 32)
        self.layer4 = nn.Linear(32, 16)
        self.layer5 - nn.
    
    def forward(self, x):
        x = torch.relu(self.input_layer(x))
        x = torch.relu(self.lyr1(x))
        x = torch.relu(self.lyr2(x))
        x = self.lyr3(x)
        return x
    
model = Chess_Model(nfeats).to(device)

In [117]:
print(df['Evaluation'].head())
z = eval_scaler.inverse_transform(y_test).astype(int)
print(z)

0   -10
1    56
2    -9
3    52
4   -26
Name: Evaluation, dtype: int64
[[ 332]
 [-110]
 [-797]
 ...
 [ -66]
 [ 150]
 [   0]]


In [23]:
def train(dataloader, model, optimizer, epoch):
    model = model.to(device)
    loss_fn = nn.MSELoss()
    model.train()
    with tqdm.tqdm(dataloader, unit="batch") as tbatch:
        for X, y in tbatch:
            X = X.to(device)
            y = y.to(device)
            optimizer.zero_grad()
            pred = model(X)
            loss = loss_fn(pred, y)
            loss.backward()
            optimizer.step()
    torch.save(
        {
            "epoch" : epoch,
            "model_state_dict" : model.state_dict(),
            "optimizer_state_dict" : optimizer.state_dict(),
            "loss" : loss
        },
        f"{CKPT_DIR}/ckpt{epoch}.pt",
    )

In [34]:
def test(dataloader, model, dataset_name):
    loss_fn = nn.MSELoss()
    model = model.to(device)
    model.eval()
    test_loss = 0.0
    correct = 0
    with torch.no_grad():
        for X, y in dataloader:
            X = X.to(device)    
            y = y.to(device)
            
            # Forward pass
            pred = model(X)
           
            # Compute loss
            curr_loss = loss_fn(pred, y).item()
            test_loss += curr_loss
            
            # Append predictions and actual values
            pred = eval_scaler.inverse_transform(pred.cpu().numpy()).astype(int)
            y = eval_scaler.inverse_transform(y.cpu().numpy()).astype(int)
            
            # Round predictions and count correct predictions
            
            correct += (pred == y).sum()
    # Compute accuracy
    accuracy = correct / len(dataloader.dataset)
    print(dataset_name)
    print(f'Accuracy: {accuracy} | Mean Squared Error: {test_loss / len(dataloader.dataset) * 1000}\n')


In [119]:
# test(train_loader, model, "TRAIN")
preds = model(torch.tensor(X_test.values, dtype=torch.float32).to(device))
preds = eval_scaler.inverse_transform(preds.cpu().detach().numpy()).astype(int)
for pred, actual, in zip(preds, z):
    print(f"Prediciton: {pred}, Actual: {actual}")


Prediciton: [57], Actual: [332]
Prediciton: [-140], Actual: [-110]
Prediciton: [69], Actual: [-797]
Prediciton: [50], Actual: [128]
Prediciton: [95], Actual: [71]
Prediciton: [53], Actual: [913]
Prediciton: [34], Actual: [90]
Prediciton: [59], Actual: [31]
Prediciton: [-21], Actual: [297]
Prediciton: [141], Actual: [0]
Prediciton: [71], Actual: [0]
Prediciton: [37], Actual: [928]
Prediciton: [-239], Actual: [72]
Prediciton: [-13], Actual: [126]
Prediciton: [169], Actual: [187]
Prediciton: [-239], Actual: [-330]
Prediciton: [-41], Actual: [-383]
Prediciton: [-44], Actual: [-358]
Prediciton: [-239], Actual: [350]
Prediciton: [104], Actual: [506]
Prediciton: [62], Actual: [15]
Prediciton: [-41], Actual: [-72]
Prediciton: [41], Actual: [-339]
Prediciton: [54], Actual: [31]
Prediciton: [72], Actual: [-12]
Prediciton: [43], Actual: [5]
Prediciton: [48], Actual: [0]
Prediciton: [83], Actual: [156]
Prediciton: [59], Actual: [50]
Prediciton: [-41], Actual: [60]
Prediciton: [48], Actual: [38]
Pr

In [25]:
def make_or_restore_model():
    model = Chess_CNN().to(device)
    optimizer = torch.optim.Adam(model.parameters(), lr=LR)
    checkpoints = [
        CKPT_DIR + "/" + name
        for name in os.listdir(CKPT_DIR)
        if name[-1] == "t"
    ]
    if checkpoints:
        latest_checkpoint = max(checkpoints, key=os.path.getctime)
        print("Restoring from", latest_checkpoint)
        ckpt = torch.load(latest_checkpoint)
        model.load_state_dict(ckpt["model_state_dict"])
        optimizer.load_state_dict(ckpt["optimizer_state_dict"])
        epoch = ckpt["epoch"]
        return model, optimizer, epoch+1
    else: 
        print("Creating new model")
        return model, optimizer, 0

In [39]:
model, optimizer, epoch_start = make_or_restore_model()
model = model.to(device)
for e in range(epoch_start, EPOCHS):
    print()
    print("Epoch", e)
    print("-------")
    model.train()
    model.to(device)
    train(train_loader, model, optimizer, e)
    print()
    model.eval()
    test(train_loader, model, "Train")
    test(test_loader, model, "Test")

Creating new model

Epoch 0
-------


100%|██████████| 10124/10124 [00:32<00:00, 312.44batch/s]



Train
Accuracy: 0.0022472603796882236 | Mean Squared Error: 0.09363615496473121

Test
Accuracy: 0.0022842890304736506 | Mean Squared Error: 0.09990957902534549


Epoch 1
-------


100%|██████████| 10124/10124 [00:32<00:00, 310.88batch/s]



Train
Accuracy: 0.0029016823583886404 | Mean Squared Error: 0.09460180107344968

Test
Accuracy: 0.002815231886205364 | Mean Squared Error: 0.10083735936223763


Epoch 2
-------


...

100%|██████████| 10124/10124 [00:33<00:00, 305.34batch/s]



Train
Accuracy: 0.0057447136903843185 | Mean Squared Error: 0.09320035123265094

Test
Accuracy: 0.005951498987504322 | Mean Squared Error: 0.09947694463838135


Epoch 84
-------


100%|██████████| 10124/10124 [00:33<00:00, 298.91batch/s]



Train
Accuracy: 0.00417965735453002 | Mean Squared Error: 0.09318386607560727

Test
Accuracy: 0.004000592680397096 | Mean Squared Error: 0.09944231868580805


Epoch 85
-------


 72%|███████▏  | 7306/10124 [00:23<00:09, 296.26batch/s]